In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# import os
# os.chdir('/content/drive/My Drive/translate')

In [ ]:
!sudo apt install rar

In [ ]:
!rar x datasets.rar

In [ ]:
!pip install transformers datasets evaluate sacrebleu nltk keras_nlp
!pip install sentencepiece

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AdamWeightDecay
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer
import datasets
import tensorflow as tf
# import keras_nlp
# from nltk.translate.bleu_score import sentence_bleu
# import evaluate
import numpy as np
import time

In [ ]:
split_char = '⫯'
data_name = 'PTT_2023_08_06'
model_name = f'mT5_small_PTT_2023_08_06_Thu_Aug_17_081645_2023.ckpt'
# model_name = f'mT5_small_{data_name}_{time.ctime().replace(" ", "_").replace(":", "")}.ckpt'
checkpoint = "google/mt5-small"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "translate engTyping to Traditional Chinese:".split()

In [ ]:


with open(f'datasets/{data_name}_engTyping_inserted_lines.txt', 'r', encoding='utf-8') as file:
    engTyping_inserted_lines = file.readlines()
with open(f'datasets/{data_name}_zh_lines.txt', 'r', encoding='utf-8') as file:
    zh_lines = file.readlines()

engTyping_inserted_lines = [prefix + line.split(split_char) for line in engTyping_inserted_lines]
lines_len = len(engTyping_inserted_lines)
train_engTyping_inserted_lines = engTyping_inserted_lines[:int(lines_len * 0.7)]
val_engTyping_inserted_lines = engTyping_inserted_lines[int(lines_len * 0.7):int(lines_len * 0.85)]
test_engTyping_inserted_lines = engTyping_inserted_lines[int(lines_len * 0.85):]
zh_lines = [line.split(split_char)[1:-1] for line in zh_lines]
lines_len = len(zh_lines)
train_zh_lines = zh_lines[:int(lines_len * 0.7)]
val_zh_lines = zh_lines[int(lines_len * 0.7):int(lines_len * 0.85)]
test_zh_lines = zh_lines[int(lines_len * 0.85):]
assert len(train_engTyping_inserted_lines) == len(train_zh_lines)
assert len(val_engTyping_inserted_lines) == len(val_zh_lines)
assert len(test_engTyping_inserted_lines) == len(test_zh_lines)
train_lines = tokenizer(train_engTyping_inserted_lines, text_target=train_zh_lines, max_length=128, is_split_into_words=True, truncation=True)
val_lines = tokenizer(val_engTyping_inserted_lines, text_target=val_zh_lines, max_length=128, is_split_into_words=True, truncation=True)
test_lines = tokenizer(test_engTyping_inserted_lines, text_target=test_zh_lines, max_length=128, is_split_into_words=True, truncation=True)

train_dataset = datasets.Dataset.from_dict(train_lines)
val_dataset = datasets.Dataset.from_dict(val_lines)
test_dataset = datasets.Dataset.from_dict(test_lines)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
model.load_weights('models/mT5_small_PTT_2023_08_06_Thu_Aug_17_081645_2023.ckpt')

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


tf_train_set = model.prepare_tf_dataset(
    # dataset["train"],
    train_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

tf_val_set = model.prepare_tf_dataset(
    # dataset["validation"],
    val_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    # dataset["validation"],
    test_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
tf.debugging.disable_traceback_filtering()

In [ ]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self, tokenizer, name="bleu", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tokenizer = tokenizer
        self.bleu = sentence_bleu
        self.bleu_score = self.add_weight(
            shape=(),
            initializer='zeros',
            dtype=self.dtype,
            name = 'bleu_score'
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        # print(dir(y_true))
        # print(dir(y_pred))
        # print(y_true.value_index)
        # print(y_pred.value_index)
        # print(y_true[2])
        # print(y_pred[0])
        # print(dir(y_true[2]))
        # print(dir(y_pred[0]))
        # print(y_true[2].value_index)
        # print(y_pred[0].value_index)
        # print(y_true[2][0])
        # print(y_pred[0][0])
        # print(y_true[2][0][0])
        # print(y_pred[0][0][0])
        # decoded_pred = self.tokenizer.batch_decode(y_pred, skip_special_tokens=True)
        # y_true = np.where(y_true != -100, y_true, tokenizer.pad_token_id)
        # decoded_true = self.tokenizer.batch_decode(y_true, skip_special_tokens=True)
        # decoded_pred, decoded_true = postprocess_text(decoded_pred, decoded_true)
        result = self.bleu(y_true, y_pred)
        self.bleu_score.assign(result)

    def result(self):
        return self.bleu_score

    def reset_state(self):
        self.bleu_score.assign(0.0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "tokenizer": self.tokenizer,
            }
        )
        return config

In [ ]:
tf_train_set.save(f'datasets/mT5_{data_name}_tf_train_dataset.tfrecord')
tf_val_set.save(f'datasets/mT5_{data_name}_tf_val_dataset.tfrecord')
# tf_test_set.save(f'datasets/mT5_{data_name}_tf_test_dataset.tfrecord')

In [ ]:
tf_train_set = tf.data.Dataset.load(f'datasets/mT5_{data_name}_tf_train_dataset.tfrecord')
tf_val_set = tf.data.Dataset.load(f'datasets/mT5_{data_name}_tf_val_dataset.tfrecord')

In [ ]:
os.chdir('../../../.')

In [ ]:
!dir

In [ ]:
drive.flush_and_unmount()

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.compile(optimizer=optimizer, metrics=["accuracy"])  # No loss argument!

# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_set)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(f'models/{model_name}',save_best_only=True, save_weights_only=True),
    tf.keras.callbacks.TensorBoard(log_dir=f'logs/{model_name}_logs')
]

model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=1, callbacks=callbacks)
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/translate')
model.save_pretrained(f'models/{model_name}')

In [ ]:
tf.keras.utils.plot_model(model, 'model.png')

In [ ]:
model.save(f'models/{model_name}', save_format='tf')

In [ ]:
split_char = '⫯'
punctuations = '、，。？！：；'

def engTyping_insert_split_char(sentence: str, split_char: str) -> str:
    insert_times = 0
    sentence_list = list(sentence)
    for i, char in enumerate(sentence):
        if char in ' 6347' + punctuations:
            sentence_list.insert(i + insert_times + 1, split_char)
            insert_times += 1
    return ''.join(sentence_list[:-1])

In [ ]:
inputs = tokenizer(prefix + engTyping_insert_split_char(input('?:'), split_char).split(split_char), is_split_into_words=True, return_tensors="tf", truncation=True).input_ids
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))